In [ ]:
# raw data를 전처리 후 일정한 갯수로 나눠 분할

import os

from preprocess import load_data, preprocess
from batch.create_batch_jsonl import generate_jsonl_batches

from settings import S3_FILE_PATH

df = load_data(S3_FILE_PATH)
df = preprocess(df)

prompt_batch_size = 1
jsonl_batch_size=6000

generate_jsonl_batches(df, prompt_batch_size=prompt_batch_size, jsonl_batch_size=jsonl_batch_size)

already exists in './raw_output.json'


100%|██████████| 96323/96323 [00:14<00:00, 6507.83it/s]


jsonl_data length:  96323
✅ Saved input_batches/batch_part_1.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_2.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_3.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_4.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_5.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_6.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_7.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_8.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_9.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_10.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_11.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_12.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_13.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_14.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_15.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_16.jsonl (6000개 요청)
✅ Saved input_batches/batch_part_17.jsonl (323개 요청)


In [ ]:
# 파일 리스트 확인

from openai import OpenAI
from settings import OPENAI_TOKEN, OPENAI_BATCH_INPUT_PATH
import os

client = OpenAI(api_key=OPENAI_TOKEN)
[f for f in os.listdir(OPENAI_BATCH_INPUT_PATH) if os.path.isfile(os.path.join(OPENAI_BATCH_INPUT_PATH, f))]

['batch_part_3.jsonl',
 'batch_part_17.jsonl',
 'batch_part_15.jsonl',
 'batch_part_1.jsonl',
 'batch_part_5.jsonl',
 'batch_part_11.jsonl',
 'batch_part_13.jsonl',
 'batch_part_7.jsonl',
 'batch_part_14.jsonl',
 'batch_part_2.jsonl',
 'batch_part_16.jsonl',
 'batch_part_12.jsonl',
 'batch_part_6.jsonl',
 'batch_part_4.jsonl',
 'batch_part_10.jsonl',
 'batch_part_8.jsonl',
 'batch_part_9.jsonl']

In [ ]:
# 배치 생성

OPENAI_BATCH_INPUT_PATH="input_batches"
file_path = f'{OPENAI_BATCH_INPUT_PATH}/batch_part_15.jsonl'

batch_input_file = client.files.create(
  file=open(file_path, "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

Batch(id='batch_6840298459f08190857e0a5774655557', completion_window='24h', created_at=1749035396, endpoint='/v1/chat/completions', input_file_id='file-XPW8hpEX33bNufQrmdg6hq', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1749121796, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
batch_output_dir_path = "./output_batches"

import pandas as pd

from batch.postprocess_batch_jsonl import list_files_in_directory, concat_jsonl_files, merge_model_output
from preprocess import load_data, preprocess
from settings import S3_FILE_PATH

output_file_list = list_files_in_directory(batch_output_dir_path)
model_output = concat_jsonl_files(output_file_list)

output_df = pd.DataFrame(
    [
        {
            "post_id": item["post_code"],
            "board_id": item["board_code"],
            **item["probs"],
        }
        for item in model_output
    ]
)

df = load_data(S3_FILE_PATH)
input_df = preprocess(df)

df = merge_model_output(input_df, output_df)

already exists in './raw_output.json'


In [12]:
df

,board_id,post_id,text,연애,취업,감정,가족,고민,친구,기숙사,...,추천,통학,축제,장소,학업,졸업,거래,생활,유머,수강신청
0,370451,293745286,디대 만화용 아이패드 어떤거 사야 좋을까요 통장에 마넌 있고 노력하면 만원까진 쓸 ...,0.0,0.0,0.0,0.0,0.3,0.0,0.0,...,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
1,370451,293744265,생산운영 출결 어떻게 진행 되는 건가요,0.0,0.0,0.0,0.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0
2,370451,293750914,아 미친 잘못봐서 교수님 보고싶다는 줄,0.0,0.0,0.7,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.4,0.0
3,370451,293746446,천안에시분도착 시수업 택시타야겠지버스가능,0.0,0.0,0.0,0.0,0.3,0.0,0.0,...,0.0,0.7,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0
4,370451,293747468,여행동아리 연락온사람 안온거면 떨어진건가 일날 차발표라고 봤는데,0.0,0.0,0.1,0.0,0.7,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96318,370451,375370519,흠 오늘 뭐입지 아침엔 추울 것 같고 학교 도착하면 더울 것 같고 집 갈 땐 애매하...,0.0,0.0,0.2,0.0,0.5,0.0,0.0,...,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0
96319,370451,375370861,선배님들 저희 친누나 한 번만 도와주세요아니면 맞아 죽어요 안녕하세요 저는 중앙대학...,0.0,0.1,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.0
96320,370451,375371053,사표 교수님 월요일 강의 가는 거져,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0
96321,370451,375374082,하이쿱 학잠 받은 것 같던데 실물 예쁨,0.0,0.0,0.0,0.0,0.0,0.0,0.2,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0


In [ ]:
df.to_csv("output.csv", index=False)

In [3]:
labels = df.drop(columns=["board_id", "post_id", "text"]).columns.tolist()
label_names = df.drop(columns=["board_id", "post_id", "text"]).columns.tolist()

In [5]:
import json
with open("label_config.json", "w", encoding="utf-8") as f:
    json.dump(label_names, f, ensure_ascii=False)

In [6]:
with open("label_config.json", "r", encoding="utf-8") as f:
    label_names = json.load(f)